In [14]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

LOCAL = False

Bilboard_ES = Elasticsearch(hosts="http://localhost", port= 9200)

In [15]:
Bilboard_ES.ping()

True

In [22]:
def generate_data(documents):
    """
    Génération des données cherchées
    """
    for docu in documents:
        yield {
            "_index": "artist",
            "_type": "artist",
            "_source": {k:v if v else None for k,v in docu.items()},
        }

In [7]:
import pandas as pd

In [24]:
data = pd.read_csv("lien.csv")
data.head()
data.columns

Index(['artist', 'title', 'lyrics'], dtype='object')

In [26]:
documents = data.fillna("").to_dict(orient="records")
bulk(Bilboard_ES, generate_data(documents))

(5492, [])

In [38]:
def search_singer(singer):
    QUERY = {
      "query": {
        "term" : { 
            "artist" : singer.lower()} 
      }
    }
    result = Bilboard_ES.search(index="artist", body=QUERY)
    source = result["hits"]["hits"]
    seen = set()
    new_source = []
    for d in source:
        t = tuple(d["_source"].items())
        if t not in seen:
            seen.add(t)
            new_source.append(d)
    
    artist = [elt['_source']['artist'] for elt in new_source]
    title = [elt['_source']['title'] for elt in new_source]
    lyrics = [elt['_source']['lyrics'] for elt in new_source]



    return title
search_singer('Eminem')

['Gnat',
 'The Real Slim Shady',
 'The Way I Am',
 'Without Me',
 'Cleanin’ Out My Closet',
 'Lose Yourself',
 'Superman',
 'Sing for the Moment',
 'Just Lose It',
 'Like Toy Soldiers']